In [3]:
print("hello")
bytes

hello


bytes

In [1]:
import os
import json

from Work.Helper import little_endian_to_int,list_all_tx,int_to_little_endian

all_files= list_all_tx() 

    
   
   
bytes

bytes

In [5]:
scriptpubkey_types=set()

for tx in all_files:
  for section in ["vin", "vout"]:
    for item in tx[section]:
        if section == "vin":     
          scriptpubkey_types.add(item["prevout"]["scriptpubkey_type"])
        else:  
           scriptpubkey_types.add(item["scriptpubkey_type"])
scriptpubkey_types  
          


{'bare-multisig',
 'op_return',
 'p2pkh',
 'p2sh',
 'v0_p2wpkh',
 'v0_p2wsh',
 'v1_p2tr'}

In [6]:
# op_return -> 
op_return_in_input=set()
op_return_in_output=set()



for tx in all_files:
  for section in ["vin", "vout"]:
    for item in tx[section]:
        if section == "vin":  
          if (item["prevout"]["scriptpubkey_type"])=='op_return':
             op_return_in_input.add(item["prevout"]["value"])
             
          scriptpubkey_types.add(item["prevout"]["scriptpubkey_type"])
        else:
            if (item["scriptpubkey_type"])=='op_return':
             op_return_in_output.add(item["value"])
               
print(op_return_in_input)
print(op_return_in_output)


set()
{0}


In [7]:
# is 0 value in vin -> 
count=0
for tx in all_files:
  for item in tx["vin"]:            
          if (item["prevout"]["value"])==0:
             count+=1
             
print(count) 



0


### No value 0 in input field


#### Investigating `unknown` keyword in `scriptpubkey-type`:


In [8]:
### First we will check that these keyword are present in both inputs and outputs field or not> 


unknown_keyword= {
   "vin":0,
   "vout":0
}

unique_commands_for_unknown=set()
trial=set()

for tx in all_files:
  for section in ["vin", "vout"]:
    for item in tx[section]:
        if section == "vout" and  item["scriptpubkey_type"]=="unknown":     
          unknown_keyword["vout"]+=1
        if section == "vin" and  item["prevout"]["scriptpubkey_type"]=="unknown":  
            unknown_keyword["vin"]+=1

        #### In some places , "unknown" keyword is written inside the scriptpubkey_type 
        ### -> but it is actually Multisig as shown ->
        
        
        if section == "vout":
           if item["scriptpubkey_type"]=="bare-multisig":
              unique_commands_for_unknown.add(item["scriptpubkey_asm"].split()[-1])
              trial.add(item["scriptpubkey"][-2:])
              

print("unknown keyword present : {}".format(unknown_keyword))   
print("unique_commands for the unknown type at the end {}".format(unique_commands_for_unknown))
print(trial)


unknown keyword present : {'vin': 0, 'vout': 0}
unique_commands for the unknown type at the end {'OP_CHECKMULTISIG'}
{'ae'}


In [9]:
for file in all_files:
    for output in file["vout"]:
        script_type = output["scriptpubkey_type"]
        if script_type == "unknown":
            output["scriptpubkey_type"] = "multisig"


In [10]:
### Thus replace `unknown` keyword to `bare-multisig` scriptpubkey-type

folder_path = "./mempool"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    for output in data["vout"]:
        script_type = output["scriptpubkey_type"]
        if script_type == "unknown":
            output["scriptpubkey_type"] = "bare-multisig"
    
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)


---


#### Version && Locktime(absolute)->


In [11]:
## how many types of version are used ->
version={
    1:0,
    2:0,
    }

seq_for_1={}

locktime_1={
    "Block":set(),
    "Time":set()
}
max_value = (1 << 32) - 1
for file in all_files:
    version[(file["version"])]+=1

    if file["version"]==1:
        for input in file["vin"]:
            if seq_for_1.get(input["sequence"])==None:
                value= input["sequence" ]
                seq_for_1.update({value:0})
            seq_for_1[value]+=1

    locktime_value= file["locktime"]
    if locktime_value<500_000_000:
       locktime_1["Block"].add(locktime_value)
       
    else:
         locktime_1["Time"].add(locktime_value)         

sorted_dict = dict(sorted(seq_for_1.items(), key=lambda item: item[0]))

print(version)
print(sorted_dict)

print("locktime for version 1: {}".format(locktime_1))

# print(version.get("helo"))
print(max(locktime_1["Block"])) 
print(max(locktime_1["Time"])) 





{1: 2259, 2: 5872}
{0: 32, 1: 29, 4443: 475, 4994: 34, 86695: 15, 160346: 418, 1516297: 220, 2559669: 4, 5860418: 8, 6746720: 360, 6777465: 178, 11617556: 5, 11851132: 12, 11910301: 513, 12311610: 220, 13424498: 17, 19526132: 232, 20230627: 3, 21176520: 20, 41617237: 15, 79593512: 13, 132648800: 8, 268435456: 29, 338499383: 21, 357913941: 349, 469337440: 42, 536870911: 442, 651726258: 69, 652251065: 2, 715827882: 57, 724759630: 659, 751167129: 9, 756373883: 8, 910345399: 20, 929525068: 34, 1073741822: 11, 1073745840: 85, 1078771002: 458, 1115905731: 11, 1227143699: 12, 1288490187: 220, 1407676897: 468, 1524582426: 1, 1601075568: 822, 1610612733: 10, 1610613036: 6, 1610616404: 344, 1656665503: 49, 1762298852: 20, 1798317181: 3, 1874281885: 28, 1879051786: 321, 1927488287: 135, 1986185223: 265, 2147483644: 79, 2147483648: 53, 2147483649: 5, 2147483667: 9, 2147483669: 380, 2147483680: 2, 2147483684: 216, 2147483687: 5, 2147483695: 2, 2147483705: 34, 2147483718: 89, 2147483720: 469, 214748

### Observation->

- Two version 1 and 2 are used
- Absolute Locktime given in field in each tx -> is passed at the current -> thus all txs are unlocked to be included in the block (from abosolute locktime perpective)

---


- Check that wether there any tx exists which do not have any input or outputs->
- If txId is of 32 bytes or not.
-


In [12]:
vin_empty=0
vout_empty=0
txid_irregular_length=set()
invalid_vout=0
invalid_value=0
empty_prevout=0
scriptpubkey_empty=0
scriptpubkey_asm_empty=0
wrong_sequence_value_count=0

seq_max_value = 2**32 - 1

for file in all_files:
    if len(file["vin"])==0:
        vin_empty+=1
    if len(file["vout"])==0:
        vout_empty+=1 
    
    for input in file["vin"]:

      if len(input["txid"]) != 64:
        txid_irregular_length.add(file["txid"])

    
      if input["prevout"]["value"] <0 or input["prevout"]["value"] > 2100000000000000:
         invalid_value+=1

      if input["vout"]<0:
         invalid_vout+=1


      if not bool(input["prevout"]):
         empty_prevout+=1


      if len(input["prevout"]["scriptpubkey"]) ==0:
         scriptpubkey_empty+=1

      if len(input["prevout"]["scriptpubkey_asm"]) ==0:
         scriptpubkey_asm_empty+=1   

      if    input["sequence"] > seq_max_value or input["sequence"]<0:
         wrong_sequence_value_count+=1
         
         
    for output in file["vout"]:
       if output["value"] <0 or output["value"] > 2100000000000000:
         invalid_value+=1    


       if len(output["scriptpubkey_asm"]) ==0:
         scriptpubkey_asm_empty+=1    


print("vin empty: {}".format(vin_empty))  
print("vout empty: {}".format(vout_empty)) 
print("txid irregular lenght:{}".format(txid_irregular_length))
print("invalid value :{}".format(invalid_value))
print("invalid vout value count:{}".format(invalid_vout))
print("prevout empty: {}".format(empty_prevout)) 
print("scriptpubkey_empty: {}".format(scriptpubkey_empty)) 
print("scriptpubkey_asm_empty: {}".format(scriptpubkey_asm_empty)) 
print("wrong_sequence_value_count: {}".format(wrong_sequence_value_count)) 




vin empty: 0
vout empty: 0
txid irregular lenght:set()
invalid value :0
invalid vout value count:0
prevout empty: 0
scriptpubkey_empty: 0
scriptpubkey_asm_empty: 0
wrong_sequence_value_count: 0


#### What is difference between scriptpubkey_address and bytes in scriptpubkey_asm ?

- `scriptpubkey_address`-> address each of script
- Bytes in asm -> use that to find the address and then match


##### Basic Sanity check over:-----------------------------------------------------------------------


### Check that whether scriptpubkey and scriptpubkey_asm matches or not ->


Sequence->

- check whether the value is in the range of 4 bytes value or not

- For version 1 -> no meaning of its value
- For version 2 -> check that whether there are tx which enables relative timelock & RBF
- Ignore `Absolute Timelock` -> as all tx in mempool are passed.


In [13]:
# for file in all_files:
#     if file["version"]== 2:
        

In [14]:
# hex_string = "5120727da28731cccc6f227d0200b43c384ae06e717f6f14f62f31145a8cbb145048"
# hex_string_without_prefix = hex_string[2:] if hex_string.startswith("0x") else hex_string
# ascii_string = bytes.fromhex(hex_string_without_prefix).decode('utf-8')

# print(ascii_string)


# stream = BytesIO(bytes.fromhex('4d04ffff001d0104455468652054696d65732030332f4a616e2f32303039204368616e63656c6c6f72206f6e206272696e6b206f66207365636f6e64206261696c6f757420666f722062616e6b73'))
# s = Script.parse(stream)
# stream

In [40]:
hex_string = "5221023d7d58a83b655617b571c1891db53642eb223164469f88b5a7ceaa7dffa0b0682103a553e30733d7a8df6d390d59cc136e2c9d9cf4e808f3b6ab009beae68dd60822210327bc7b5a44643a3225f4a0fde3d0c0becefaedf33952d0499b8da55e542f79a953ae"

# len(bytes.fromhex(hex_string))
print(len(hex_string))
# int('69', 16)
inter= bytes.fromhex(hex_string)
inter= bytes.fromhex('a2')


print(int.from_bytes(inter,'little'))


print(int.from_bytes(inter, 'big'))


# type(hex_string)
bytes.fromhex(hex_string)
# bytes.fromHex(hex_string)

# int_to_little_endian(105,1)

# int_to_little_endian

little_endian_to_int(bytes.fromhex(hex_string))

210
162
162


NameError: name 'little_endian_to_int' is not defined

In [16]:
type(bytes)
bytes.fromhex(hex_string)

b'R!\x02=}X\xa8;eV\x17\xb5q\xc1\x89\x1d\xb56B\xeb"1dF\x9f\x88\xb5\xa7\xce\xaa}\xff\xa0\xb0h!\x03\xa5S\xe3\x073\xd7\xa8\xdfm9\rY\xcc\x13n,\x9d\x9c\xf4\xe8\x08\xf3\xb6\xab\x00\x9b\xea\xe6\x8d\xd6\x08"!\x03\'\xbc{ZDd:2%\xf4\xa0\xfd\xe3\xd0\xc0\xbe\xce\xfa\xed\xf39R\xd0I\x9b\x8d\xa5^T/y\xa9S\xae'

- Are there any tx which uses input from the coinbase tx


In [17]:
bool_value=set()
for file in all_files:
    for input in file["vin"]:       
     bool_value.add(input["is_coinbase"])
bool_value    

{False}

In [ ]:
for file in all_files:
    for input in file["vin"]:
        prevout=input["prevout"]
        if prevout["scriptpubkey_type"] is in ['v0_p2wpkh','v0_p2wsh','v1_p2tr']:
             if input["scriptsig"] !="" and input["scriptsig_asm"]!="" and input["witness"] == "":
                 
            

In [6]:
my_dict= {
    1:0,
    2:8
}

# my_dict[3]
my_dict.get(3)==None

True

set()